# Text Feature Engineering - No LSA

In [1]:
from __future__ import print_function

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()

from sklearn.feature_extraction.text import TfidfVectorizer

%matplotlib inline

pd.options.mode.chained_assignment = None

In [6]:
train_df = pd.read_json('train.json')
train_df.head()

,bathrooms,bedrooms,building_id,created,description,display_address,features,interest_level,latitude,listing_id,longitude,manager_id,photos,price,street_address
10,1.5,3,53a5b119ba8f7b61d4e010512e0dfc85,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,[],medium,40.7145,7211212,-73.9425,5ba989232d0489da1b5f2c45f6688adc,[https://photos.renthop.com/2/7211212_1ed4542e...,3000,792 Metropolitan Avenue
10000,1.0,2,c5c8a357cba207596b04d1afd1e4f130,2016-06-12 12:19:27,,Columbus Avenue,"[Doorman, Elevator, Fitness Center, Cats Allow...",low,40.7947,7150865,-73.9667,7533621a882f71e25173b27e3139d83d,[https://photos.renthop.com/2/7150865_be3306c5...,5465,808 Columbus Avenue
100004,1.0,1,c3ba40552e2120b0acfc3cb5730bb2aa,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,"[Laundry In Building, Dishwasher, Hardwood Flo...",high,40.7388,6887163,-74.0018,d9039c43983f6e564b1482b273bd7b01,[https://photos.renthop.com/2/6887163_de85c427...,2850,241 W 13 Street
100007,1.0,1,28d9ad350afeaab8027513a3e52ac8d5,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,"[Hardwood Floors, No Fee]",low,40.7539,6888711,-73.9677,1067e078446a7897d2da493d2f741316,[https://photos.renthop.com/2/6888711_6e660cee...,3275,333 East 49th Street
100013,1.0,4,0,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,[Pre-War],low,40.8241,6934781,-73.9493,98e13ad4b495b9613cef886d79a6291f,[https://photos.renthop.com/2/6934781_1fa4b41a...,3350,500 West 143rd Street


In [16]:
main_df = pd.read_pickle('Pickled_Files/X_num.pickle')

In [17]:
main_df.head()

,bathrooms,bedrooms,price,num_photos,num_features,description_length,latitude,longitude,manager_bin_1,manager_bin_2,manager_bin_3,manager_bin_4,manager_bin_5,month_bin_1,month_bin_2,month_bin_3
0,1.5,3,3000.0,5,0,95,40.7145,-73.9425,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
1,1.0,1,2250.0,5,1,63,40.7216,-73.9423,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1.0,1,2200.0,5,2,105,40.6905,-73.9668,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1.0,1,1950.0,4,0,29,40.6963,-73.9239,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
4,1.0,1,2000.0,4,0,27,40.7056,-73.9209,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0


In [18]:
text_features_df = train_df[['features', 'description', 'listing_id']]
text_features_df.head()

,features,description,listing_id
10,[],A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,7211212
10000,"[Doorman, Elevator, Fitness Center, Cats Allow...",,7150865
100004,"[Laundry In Building, Dishwasher, Hardwood Flo...","Top Top West Village location, beautiful Pre-w...",6887163
100007,"[Hardwood Floors, No Fee]",Building Amenities - Garage - Garden - fitness...,6888711
100013,[Pre-War],Beautifully renovated 3 bedroom flex 4 bedroom...,6934781


In [19]:
features_df = text_features_df['features'].apply(lambda x: " ".join(x))

In [20]:
features_df.head()

10                                                         
10000     Doorman Elevator Fitness Center Cats Allowed D...
100004    Laundry In Building Dishwasher Hardwood Floors...
100007                               Hardwood Floors No Fee
100013                                              Pre-War
Name: features, dtype: object

In [21]:
features_df.shape

(49352,)

### Putting `features` through `TfidVectorizer`

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words='english', use_idf=True, max_df=0.20)
#ngram_range=(1, 2) - do not use ngrams as get features that do not make sense

listing_features_vectorizer = tfidf.fit_transform(features_df)

feature_array = np.array(tfidf.get_feature_names())
tfidf_sorting = np.argsort(listing_features_vectorizer.toarray()).flatten()[::-1]

n = 100
top_n = feature_array[tfidf_sorting][:n]
top_n

array([u'york', u'fees', u'entrance', u'entire', u'entertainment',
       u'entertain', u'enormous', u'end', u'enclosed', u'en', u'elev',
       u'elegant', u'elegance', u'electronic', u'electricity', u'electric',
       u'ego', u'effective', u'eat', u'easy', u'east', u'e60s', u'e50s',
       u'dyer', u'dw', u'duty', u'duplex', u'dryer', u'dry', u'drs',
       u'dressing', u'entrances', u'entryway', u'equal', u'extravagant',
       u'featuring', u'features', u'feature', u'fast', u'fans', u'fan',
       u'family', u'facing', u'facility', u'facilities', u'eye',
       u'extreme', u'extravaganza', u'extra', u'equipment', u'exquisite',
       u'exposure', u'exposed', u'expansive', u'exercise', u'executive',
       u'exclusive', u'excellent', u'excellence', u'events', u'european',
       u'eraser', u'equipped', u'drenched', u'dream', u'double',
       u'convertible', u'cpw', u'courtyard', u'courts', u'court',
       u'countertops', u'counters', u'counter', u'cost', u'corner',
       u'coord

In [23]:
listing_features_vectorizer.shape

(49352, 1029)

## Trying Feature Text without SVD

In [25]:
y = np.load('y.npy')

In [26]:
y.shape

(49352,)

In [27]:
X_feature_text = listing_features_vectorizer

In [28]:
X_feature_text.shape

(49352, 1029)

In [40]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB

from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, classification_report
from sklearn.metrics import log_loss

In [41]:
X_train, \
X_test,  \
y_train, \
y_test = train_test_split(X_feature_text, y, test_size=0.20, random_state=82)

In [42]:
def evaluate_model(model):
    
    model.fit(X_train, y_train)
    
    train_score = model.score(X_train, y_train)
    test_score = model.score(X_test, y_test)
    
    y_hat = model.predict(X_train)
    y_pred = model.predict(X_test)
    
    y_hat_proba = model.predict_proba(X_train)
    y_pred_proba = model.predict_proba(X_test)
    
    return {'model': model,
            'X_test': X_test,
            'X_train': X_train,
            'y': y,
            'y_test': y_test,
            'y_train': y_train,
            'y_hat': y_hat,
            'y_pred': y_pred,
            'y_hat_proba': y_hat_proba,
            'y_pred_proba': y_pred_proba,
            'train_score': train_score,
            'test_score': test_score
           }

In [43]:
def model_summary(model, data_dictionary):
    
    class_rpt = classification_report(data_dictionary['y_test'], data_dictionary['y_pred'])
    log_loss_score_train = log_loss(data_dictionary['y_train'], data_dictionary['y_hat_proba'])
    log_loss_score_test = log_loss(data_dictionary['y_test'], data_dictionary['y_pred_proba'])
    
    print ("Model Summary Report:\n")
    print (model)
    print ("\n")
    print ("Classification Report:\n")
    print (class_rpt)
    print ("Accuracy on training set: {:4f}".format(model.score(data_dictionary['X_train'], 
                                                                data_dictionary['y_train'])))
    
    print ("Accuracy on test set:     {:4f}".format(model.score(data_dictionary['X_test'], 
                                                            data_dictionary['y_test'])))
    print ("\n")
    print ("Log Loss on training set: {:4f}".format(log_loss_score_train))
    print ("Log Loss on test set:     {:4f}".format(log_loss_score_test))

In [44]:
clf1 = LogisticRegression(penalty='l2',
                          C=0.001,
                          n_jobs=-1,
                          random_state=82
                         )

In [45]:
clf1_dictionary = evaluate_model(clf1)

In [46]:
model_summary(clf1, clf1_dictionary)

Model Summary Report:

LogisticRegression(C=0.001, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=-1,
          penalty='l2', random_state=82, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)


Classification Report:

             precision    recall  f1-score   support

          0       0.00      0.00      0.00       765
          1       0.69      1.00      0.82      6813
          2       0.00      0.00      0.00      2293

avg / total       0.48      0.69      0.56      9871

Accuracy on training set: 0.695803
Accuracy on test set:     0.690204


Log Loss on training set: 0.798191
Log Loss on test set:     0.803665


/Users/nate_velarde/anaconda/lib/python2.7/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


## Using Naive Bayes Classifier

In [69]:
clf2 = MultinomialNB()

In [70]:
clf2_dictionary = evaluate_model(clf2)

In [71]:
model_summary(clf2, clf2_dictionary)

Model Summary Report:

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)


Classification Report:

             precision    recall  f1-score   support

          0       0.10      0.00      0.01       765
          1       0.69      0.99      0.81      6813
          2       0.21      0.00      0.01      2293

avg / total       0.53      0.69      0.56      9871

Accuracy on training set: 0.698260
Accuracy on test set:     0.686253


Log Loss on training set: 1.016213
Log Loss on test set:     1.090299


## Preparing `description` for processing

In [50]:
description_df = text_features_df['description']
description_df.head()

10        A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...
10000                                                      
100004    Top Top West Village location, beautiful Pre-w...
100007    Building Amenities - Garage - Garden - fitness...
100013    Beautifully renovated 3 bedroom flex 4 bedroom...
Name: description, dtype: object

In [72]:
description_df.values[:5]

array([ u"a brand new 3 bedroom 1 5 bath apartmentenjoy these following apartment features as you rent here  modern designed bathroom w  a deep spa soaking tub  room to room ac heat  real oak hardwood floors  rain forest shower head  ss steel appliances w  chef gas cook oven and large fridge  washer  dryer in the apt  cable internet ready  granite counter top kitchen w  lot of cabinet storage spaceit's just a few blocks to l train  don't miss out we have several great apartments in the immediate area   for additional information   ",
       u'AFULLEMPYSTRING',
       u'top top west village location, beautiful pre war building with laundry in the basement and live in super   apartment features a large bedroom with closet  separate living room, kitchen features granite tops, dishwasher and microwave included, marble bathroom and hardwood flooring  building is very well maintained and conveniently located near a,c,e,l,1,2,3 trains  surrounded by many local cafe s, restaurants, available f

## Cleaning Up Text Data Somewhat Non-Pythonically

In [52]:
description_df = description_df.str.lower()
description_df = description_df.str.replace(r'^\s+$', 'AFULLEMPYSTRING')
description_df = description_df.str.replace(r'<p><a  website_redacted ', ' ')
description_df = description_df.str.replace(r'!<br /><br />', ' ')
description_df = description_df.str.replace(r"<br', '/><br", ' ')
description_df = description_df.str.replace(r"\b(st.)\b", ' street ')
description_df = description_df.str.replace(r"\b(dr)\b", ' drive ')
description_df = description_df.str.replace(r"-", ' ')
description_df = description_df.str.replace(r"$amp;", '&')
description_df = description_df.str.replace(r"!", ' ')
description_df = description_df.str.replace(r"*", ' ')
description_df = description_df.str.replace(r"\(", ' ')
description_df = description_df.str.replace(r"\)", ' ')
description_df = description_df.str.replace(r"+", ' ')
description_df = description_df.str.replace(r"\t", ' ')
description_df = description_df.str.replace(r"\r", ' ')
description_df = description_df.str.replace(r"\b(hr)\b", ' hour')
description_df = description_df.str.replace(r"\b(w/)\b", ' with')
description_df = description_df.str.replace(r"<br />", ' ')
description_df = description_df.str.replace(r"<br/>", ' ')
description_df = description_df.str.replace(r"<br>", ' ') # can you do a wildcard to get rid of everything between <>
description_df = description_df.str.replace(r"/", ' ')
description_df = description_df.str.replace(r"\b(lg)\b", 'large')
description_df = description_df.str.replace(r"<a>", ' ')
description_df = description_df.str.replace(r"<li>", ' ')
description_df = description_df.str.replace(r"<ul>", ' ')
description_df = description_df.str.replace(r"(<.>)", ' ')

description_df = description_df.str.replace(r"\b(wi fi)\b", 'wi-fi')
description_df = description_df.str.replace(r"\b(wifi)\b", 'wi-fi')

description_df = description_df.str.replace(r"<[^>]*>", ' ')
description_df = description_df.str.replace(r"<a target=_blank  houref= website_redacted", ' ')

description_df = description_df.str.replace(r"\b(24  hour)\b", '24-hour')
description_df = description_df.str.replace(r"\b(24 hour)\b", '24-hour')
description_df = description_df.str.replace(r"\b(24hr)\b", '24-hour')

# regex for telephone numbers

# 123 456 7890
description_df = description_df.str.replace(r"\b\d{3}[\s]?\d{3}[\s]?\d{4}\b", ' ')

# 123 456 7890 + letters immediately after phone number
description_df = description_df.str.replace(r"\b\d{3}[\s]?\d{3}[\s]?\d{4}[a-z]+\b", ' ')

# letters immediately before number no space [a-z]123 456 7890
description_df = description_df.str.replace(r"\b[a-z]+\d{3}[\s]?\d{3}[\s]?\d{4}", ' ')

# 123.456.7890 or 123-456-7890
description_df = description_df.str.replace(r"\b\d{3}[-.]?\d{3}[-.]?\d{4}\b", ' ')

description_df = description_df.str.replace(r"\b(website_redacted)\b", ' ')

# regex to get rid of e-mail addresses
description_df = description_df.str.replace(r"\b(kagglemanager@renthop.com)\b", ' ')

# regex to get rid of word.com
description_df = description_df.str.replace(r"\b[a-z]+\.(com)\b", ' ')
description_df = description_df.str.replace(r"\b[a-z]+\.(net)\b", ' ')

# regex to get rid of www.
description_df = description_df.str.replace(r"\b(www.)\b", ' ')

# regex to get rid of .com
description_df = description_df.str.replace(r"\b(.com)\b", ' ')

# converting '&' to 'and' -- although this may be made moot by stop words
description_df = description_df.str.replace(r"&", 'and')

# removing question marks
description_df = description_df.str.replace(r"?", ' ')

# removing question marks
description_df = description_df.str.replace(r":", ' ')

# removing periods
description_df = description_df.str.replace(r".", ' ')

# removing 'eet'
description_df = description_df.str.replace(r"\b(eet)\b", ' ')

# removing 'pipes'
description_df = description_df.str.replace(r"|", '')

# removing '\'
description_df = description_df.str.replace(r"\\", ' ')

# removing '='
description_df = description_df.str.replace(r"=", ' ')

# removing '247'
description_df = description_df.str.replace(r"\b(247)\b", '24-hour 7-day')

# removing '[]' 
description_df = description_df.str.replace(r"\[", ' ')
description_df = description_df.str.replace(r"\]", ' ')

# removing 'web id #'
description_df = description_df.str.replace(r"\b(web id #)\b", ' ')

# removing 'contact'
description_df = description_df.str.replace(r"\b(contact)\b", ' ')
                                            
# removing 'e-mail'
description_df = description_df.str.replace(r"\b(e-mail)\b", ' ')                                            
description_df = description_df.str.replace(r"\b(e mail)\b", ' ')
description_df = description_df.str.replace(r"\b(email)\b", ' ')

# removing 'call and text'
description_df = description_df.str.replace(r"\b(call)\b", ' ')                                            
description_df = description_df.str.replace(r"\b(text)\b", ' ')
description_df = description_df.str.replace(r"\b(phone)\b", ' ')

# removing more tags
description_df = description_df.str.replace(r"<a target_blank href", ' ')

# removing more tags
description_df = description_df.str.replace(r";", ' ')

# removing '1000s'
description_df = description_df.str.replace(r"\b(1000s)\b", ' ')

# removing '@'at
description_df = description_df.str.replace(r"@", 'at')

# removing 'website'
description_df = description_df.str.replace(r"\b(website)\b", ' ')

# removing 'www'
description_df = description_df.str.replace(r"\b(www)\b", ' ')

# removing '_ and ~'
description_df = description_df.str.replace(r"_", ' ')
description_df = description_df.str.replace(r"~", ' ')

In [53]:
description_df.values[200:300]

array([ u'this pre war building located in the upper east side is pet friendly, has elevator and voice intercom  ',
       u"this spacious one bedroom apartment is located in manhattan's kips bay neighborhood  this elevator building has a 24-hour doorman, roof deck , and laundry  the 4, 5, 6, n, and r subway lines are all within walking distance  pictures are of a standard unit                ",
       u'make this stunning true 3 bedroom 2 bathroom apartment your new home  this gut renovated apartment features a beautiful open kitchen  living room layout as you enter the apartment divided by a granite counter top island, accompanied by white cabinetry, stainless steel appliances and inclusive of a dishwasher  each of the bedrooms can at least fit a queen size bed and perhaps a king size bed in perhaps 2 of the bedrooms  one of the bedrooms has an en suite bathroom with a stand up shower  all bedrooms has full closets as well as beautiful large windows allowing for lots of natural light

In [54]:
type(description_df)

pandas.core.series.Series

In [55]:
description_df.shape

(49352,)

### Putting `description` through `TfidVectorizer`

In [56]:
tfid_description_df = description_df
tfid_description_df.shape

(49352,)

In [57]:
tfidf = TfidfVectorizer(stop_words='english', use_idf=True, max_df=0.20)
#ngram_range=(1, 2) - do not use ngrams as get features that do not make sense, try it without stopwords

description_vectorizer = tfidf.fit_transform(tfid_description_df)

In [58]:
tfidf.get_feature_names()[0:20]

[u'00',
 u'000',
 u'0000',
 u'000guarantors',
 u'000july',
 u'000large',
 u'000may',
 u'000s',
 u'000sf',
 u'000sft',
 u'000sq',
 u'000sqft',
 u'0058',
 u'0068',
 u'0079',
 u'0085',
 u'0090',
 u'00am',
 u'00apartment',
 u'00call']

In [59]:
len(tfidf.get_feature_names())

34784

## Description Feature Without SVD

In [61]:
X_description_text = description_vectorizer

In [62]:
X_train, \
X_test,  \
y_train, \
y_test = train_test_split(X_description_text, y, test_size=0.20, random_state=82)

In [63]:
clfd1 = LogisticRegression(penalty='l2',
                          C=0.001,
                          n_jobs=-1,
                          random_state=82
                         )

In [64]:
clfd1_dictionary = evaluate_model(clfd1)

In [65]:
model_summary(clfd1, clfd1_dictionary)

Model Summary Report:

LogisticRegression(C=0.001, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=-1,
          penalty='l2', random_state=82, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)


Classification Report:

             precision    recall  f1-score   support

          0       0.00      0.00      0.00       765
          1       0.69      1.00      0.82      6813
          2       0.00      0.00      0.00      2293

avg / total       0.48      0.69      0.56      9871

Accuracy on training set: 0.695803
Accuracy on test set:     0.690204


Log Loss on training set: 0.797898
Log Loss on test set:     0.803310


## Using Naive Bayes Classifier on Description Text

In [66]:
clfd2 = MultinomialNB()

In [67]:
clfd2_dictionary = evaluate_model(clfd2)

In [68]:
model_summary(clf2, clf2_dictionary)

Model Summary Report:

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)


Classification Report:

             precision    recall  f1-score   support

          0       0.08      0.01      0.01       765
          1       0.69      0.99      0.81      6813
          2       0.38      0.00      0.01      2293

avg / total       0.57      0.69      0.56      9871

Accuracy on training set: 0.694435
Accuracy on test set:     0.686962


Log Loss on training set: 0.792392
Log Loss on test set:     0.803180
